<a href="https://colab.research.google.com/github/soaeng/recomSys/blob/main/MF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 SGD를 사용한 MF 기본 알고리즘

In [3]:
import os
import numpy as np
import pandas as pd

base_src = 'drive/MyDrive/RecoSys/Data'
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep = '\t', names = r_cols, encoding = 'latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

In [20]:
# @title MF 클래스화

class MF() :
  def __init__(self, ratings, hyper_params):
    self.R = np.array(ratings)
    self.num_users, self.num_items = np.shape(self.R)
    self.K = hyper_params['K']
    self.alpha = hyper_params['alpha']
    self.beta = hyper_params['beta']
    self.iterations = hyper_params['iterations']
    self.verbose = hyper_params['verbose']

  def rmse(self):
    xs, ys = self.R.nonzero()
    self.predictions = []
    self.errors = []

    for x, y in zip(xs, ys):
      prediction = self.get_prediction(x, y)
      self.predictions.append(prediction)
      self.errors.append(self.R[x, y] - prediction)
    self.predictions = np.array(self.predictions)
    self.errors = np.array(self.errors)

    return np.sqrt(np.mean(self.errors ** 2))

  def train(self):
    self.P = np.random.normal(scale = 1. / self.K, size = (self.num_users, self.K))
    self.Q = np.random.normal(scale = 1. / self.K, size = (self.num_items, self.K))
    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])

    rows, columns = self.R.nonzero()
    self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

    training_process = []

    for i in range(self.iterations):
      np.random.shuffle(self.samples)
      self.sgd()
      rmse = self.rmse()
      training_process.append((i + 1, rmse))
      if self.verbose:
        if (i + 1) % 10 == 0:
          print('Iteration: %d; train RMSE = %.4f' %(i+1, rmse))

    return training_process

  def get_prediction(self, i, j):
    prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j,].T)
    return prediction

  def sgd(self):
    for i, j, r in self.samples:
      prediction = self.get_prediction(i, j)
      e = (r - prediction)

      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

      self.P[i,:] += self.alpha * ((e * self.Q[j,:]) - (self.beta * self.P[i,:]))
      self.Q[j,:] += self.alpha * ((e * self.P[i,:]) - (self.beta * self.Q[j,:]))


In [21]:
# @title MF 적용 결과

R_temp = ratings.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)

hyper_params = {
    'K': 30,
    'alpha': 0.001,
    'beta': 0.02,
    'iterations': 100,
    'verbose': True
}

mf = MF(R_temp, hyper_params)

train_process = mf.train()

Iteration: 10; train RMSE = 0.9585
Iteration: 20; train RMSE = 0.9374
Iteration: 30; train RMSE = 0.9281
Iteration: 40; train RMSE = 0.9225
Iteration: 50; train RMSE = 0.9183
Iteration: 60; train RMSE = 0.9143
Iteration: 70; train RMSE = 0.9096
Iteration: 80; train RMSE = 0.9030
Iteration: 90; train RMSE = 0.8937
Iteration: 100; train RMSE = 0.8816
